In [1]:
import keras
import random
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.random.seed(42) # NumPy
random.seed(42) # Python
tf.random.set_seed(42) # Tensorflow

In [2]:
train_data = pd.read_pickle('final_train_data.pkl')

# Feature selection
X = train_data.drop(['isFraud'], axis=1)
y = train_data['isFraud']

In [3]:
from sklearn.feature_selection import mutual_info_classif, SelectKBest

selector = SelectKBest(mutual_info_classif, k=50)
X_reduced = selector.fit_transform(X, y)
X_reduced.shape

cols = selector.get_support(indices=True)
selected_columns = X.iloc[:,cols].columns.tolist()
selected_columns

selected_columns.append('TransactionAmt')

# Get the mutual information scores
mutual_info_scores = selector.scores_

# Create a DataFrame to display the feature names and their corresponding scores
feature_scores = pd.DataFrame({'Feature': X.columns, 'Mutual Info Score': mutual_info_scores})
feature_scores = feature_scores.sort_values(by='Mutual Info Score', ascending=False)

In [4]:
feature_scores

,Feature,Mutual Info Score
9,addr2,0.134843
170,os_version,0.132732
147,id_20,0.125712
153,id_36,0.124723
151,id_33,0.124213
155,DeviceType,0.120813
7,card6,0.114477
143,id_15,0.104775
148,id_28,0.103391
149,id_29,0.102786


In [5]:
feature_scores.to_csv('feature_scores.csv', index=False)

In [6]:
selected_columns

['ProductCD',
 'card3',
 'card4',
 'card6',
 'addr2',
 'R_emaildomain',
 'D12',
 'D13',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M7',
 'M8',
 'M9',
 'V18',
 'V30',
 'V40',
 'V60',
 'V70',
 'V80',
 'V81',
 'V91',
 'V173',
 'V174',
 'V223',
 'id_12',
 'id_13',
 'id_15',
 'id_16',
 'id_17',
 'id_19',
 'id_20',
 'id_28',
 'id_29',
 'id_30',
 'id_33',
 'id_35',
 'id_36',
 'id_37',
 'DeviceType',
 'DeviceInfo',
 'R_parent_domain',
 'R_domain_name',
 'R_top_level_domain',
 'device_name',
 'device_version',
 'os_name',
 'os_version',
 'TransactionAmt']